In [1]:

# Import necessary libraries for data manipulation, plotting, and network analysis
import copy
import networkx as nx  # For handling graph data structures
import pandapower as pp  # For power system modeling
import pandapower.topology as top  # For topology analysis in Pandapower
import pandapower.plotting as plot  # For plotting in Pandapower
import GridCalEngine.api as gce  # For interfacing with the GridCal API
from GridCalEngine.Compilers.circuit_to_newton_pa import translate_newton_pa_pf_results, newton_pa_pf
from GridCalEngine.IO.file_handler import FileOpen, FileSave
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation using DataFrames
import logging  # For logging messages
import random  # For generating random numbers
from collections import Counter  # For counting hashable objects
import simbench as sb
import numpy as np

import os
os.getcwd()

import sys
sys.path.append("D:/15_Thesis-code/DistributionNetwork_libraries/")


In [2]:
class Panda2GridCal:

    def __init__(self, pandanet, Sbase=100):
        # Create a new GridCal multi-circuit grid instance
        self.grid = gce.MultiCircuit()
        self.pandanet = pandanet    
        self.Sbase = Sbase

    def ImportBuses(self):
        # Add buses to the GridCal grid based on Pandapower data
        self.bus_dictionary = dict()
        for _, bus in self.pandanet.bus.iterrows():
            newbus = gce.Bus(bus['name'], Vnom=bus['vn_kv'], vmin=bus['min_vm_pu'], vmax=bus['max_vm_pu'],active=bus['in_service'])  # Create a bus with nominal voltage
            self.grid.add_bus(newbus)  # Add the bus to the GridCal grid
            self.bus_dictionary[bus.name] = newbus

    def ImportExtGrid(self):
        # Add external grid (slack bus) generators to the GridCal grid
        gen_dictionary = dict()
        for _, gen in self.pandanet.ext_grid.iterrows():
            bus = self.bus_dictionary[gen['bus']]      
            newgen = gce.Generator(gen['name'], vset=gen['vm_pu'], Qmin=-9999, Qmax=9999, Snom=9999, P=1.0)
            self.grid.add_generator(bus, newgen)  # Add generator to the grid
            gen_dictionary[gen['name']] = newgen

    def ImportLoads(self):
        # Add loads to the GridCal grid based on Pandapower data
        load_dictionary = dict()
        for _, load in self.pandanet.load.iterrows():
            bus = self.bus_dictionary[load['bus']]
            newload = gce.Load(load['name'], P=load['p_mw'], Q=load['q_mvar'])
            self.grid.add_load(bus, newload)  # Add generator to the grid
            load_dictionary[load['name']] = newload

    def ImportShunts(self):
        # Add shunt elements (capacitors or inductors) to the GridCal grid
        # Note: Pandapower works with power, while GridCal works with admittances
        shunt_dictionary = dict()
        for _, shunt in self.pandanet.shunt.iterrows():
            bus = self.bus_dictionary[shunt['bus']]
    
            zbase = (self.pandanet.bus.loc[shunt['bus'],'vn_kv']) ** 2 / self.Sbase  # Calculate base impedance
            gu = shunt.p_mw / shunt.vn_kv ** 2 * zbase
            bu = shunt.q_mvar / shunt.vn_kv ** 2 * zbase
            
            newshunt = gce.Shunt(shunt['name'], G=gu, B=bu)
            self.grid.add_shunt(bus, newshunt)  # Add generator to the grid
            shunt_dictionary[shunt['name']] = newshunt

    def ImportLines(self):
        # Add lines (conductors) to the GridCal grid
        line_dictionary = dict()
        for idx, line in self.pandanet.line.iterrows():
            bus1 = self.bus_dictionary[line['from_bus']]
            bus2 = self.bus_dictionary[line['to_bus']]
    
            zbase = (self.pandanet.bus.loc[line['from_bus'],'vn_kv']) ** 2 / self.Sbase  # Calculate base impedance
            ru = line['r_ohm_per_km'] * line['length_km'] / zbase
            xu = line['x_ohm_per_km'] * line['length_km'] / zbase
            cu = 2*np.pi*self.pandanet.f_hz*line['c_nf_per_km'] * line['length_km']* 1e-9 * zbase
            enabled = line['in_service']
            newline = gce.Line(bus1, bus2, name=line['name'], r=ru, x=xu, b=cu, active = enabled)
           
            # Uncomment the following lines if line activation status is needed
#            if (self.lines[self.lines['id'] == idx]['Enabled'].values[0] == False):
#                line.active = False
            self.grid.add_line(newline)  # Add line to the grid
            line_dictionary[line['name']] = newline
        
    def ImportImpedances(self):
        # Add impedance elements to the GridCal grid
        for _, impedance in self.pandanet.impedance.iterrows():
            bus1 = self.bus_dictionary[impedance['from_bus']]
            bus2 = self.bus_dictionary[impedance['to_bus']]
            mult = 1.6  # Multiplicative factor for impedance
    
            zbase = (self.pandanet.bus.loc[impedance['from_bus'],'vn_kv']) ** 2 / Sbase  # Calculate base impedance
            ru = impedance.rft_pu * impedance.sn_mva / zbase * mult
            xu = impedance.xft_pu * impedance.sn_mva / Zbase * mult             
            newimpedance = gce.Line(bus1, bus2, name=line['name'], r=ru, x=xu, b=0)
            self.grid.add_line(newimpedance)  # Add generator to the grid
            line_dictionary[line['name']] = newimpedance

    def ImportStorages(self):
        # Add storage devices (batteries) to the GridCal grid
        storage_dictionary = dict()
        for _, storage in self.pandanet.storage.iterrows():
            bus = self.bus_dictionary[storage['bus']]
            newstorage = gce.Battery(Pmin=storage['min_p_mw'], Pmax=storage['max_p_mw'], Qmin=storage['min_q_mvar'],
                                     Qmax=storage['max_q_mvar'], Sbase=storage['sn_mva'], Enom=storage['max_e_mwh'],
                                     active=storage['in_service'], soc=storage['soc_percent'])  # Create battery
            self.grid.add_battery(bus, newstorage)  # Add battery to the grid
            storage_dictionary[storage['name']] = newstorage

    def ImportSGens(self):
        # Add synchronous generators (sgen) to the GridCal grid
        sgen_dictionary = dict()
        for _, sgen in self.pandanet.sgen.iterrows():
            bus = self.bus_dictionary[sgen['bus']]
            newsgen = gce.Generator(name=sgen['name'], P=sgen['p_mw'], active=sgen['in_service'], is_controlled=False)  # Create generator
            self.grid.add_generator(bus, newsgen)  # Add generator to the grid
            sgen_dictionary[sgen['name']] = newsgen 
           
    def ImportTrafos(self):
        # Add transformers to the GridCal grid
        trafo_dictionary = dict()
        for _, trafo in self.pandanet.trafo.iterrows():
            bus1 = self.bus_dictionary[trafo['hv_bus']]
            bus2 = self.bus_dictionary[trafo['lv_bus']]
            newtrafo = gce.Transformer2W(bus_from=bus1, bus_to=bus2, name='Transformer 1', HV=trafo['vn_hv_kv'],
                                         LV=trafo['vn_lv_kv'], nominal_power=trafo['sn_mva'], r=0.01,x=0.1)  # Create transformer
            self.grid.add_transformer2w(newtrafo)  # Add transformer to the grid
            trafo_dictionary[trafo['name']] = newtrafo
            
    def ImportSwitches(self):
        #print("### SWITCHES")
        # Add switches to the GridCal grid
        for _, switch in self.pandanet.switch.iterrows():
            # Select the first bus for the switch from GridCal
            #print("### SWITCH:")
            GCbus1 = self.bus_dictionary[switch['bus']]
            # If the other element in Pandapower is a bus
            if (switch['et'] == 'b'):
                GCbus2 = self.bus_dictionary[switch['element']] # Find corresponding bus
            else:
                # Create an auxiliary bus to connect between the line and the switch
                vn = self.pandanet.bus.loc[switch['bus'], 'vn_kv']  # Get the nominal voltage for the auxiliary bus
                GCbus2 = gce.Bus('BusAux ' + switch['et'] + str(switch['element']), Vnom=vn )  # Create auxiliary bus
                self.grid.add_bus(GCbus2)  # Add auxiliary bus to the grid
    
                # If the other element in Pandapower is a line
                if (switch['et'] == 'l'):
                    # The line is connected between two buses
                    line_from_bus = self.pandanet.line.loc[switch['element'], 'from_bus']
                    line_to_bus = self.pandanet.line.loc[switch['element'], 'to_bus']
    
                    if (self.pandanet.line.loc[switch.element, 'from_bus'] == switch.bus):
                        # Substitute bus1 in the element by the auxiliary bus (if applicable)
                        pass
                    else:
                        # Substitute bus2 in the element by the auxiliary bus (if applicable)
                        pass
    
                    # Find the line to modify based on its connection to bus1
                    linetomodify = [row for row in self.grid.lines if (row.bus_from == GCbus1)]
                    if linetomodify:
                        ltm = linetomodify[0]  # Found a line to modify
                    else:
                        linetomodify = [row for row in self.grid.lines if (row.bus_to == GCbus1)]
                        ltm = linetomodify[0]  # Found a line to modify
            # If the other element in Pandapower is a transformer
            if (switch['et'] == 't'):
                # Create an auxiliary bus to connect between the transformer and the switch
                bus2 = gce.Bus('BusAux ' + switch['et'] + str(b['element']), Vnom=Vbase )  # Create auxiliary bus
                self.grid.add_bus(bus2)  # Add auxiliary bus to the grid
    
                # Substitute bus1 in the element by the auxiliary bus (if applicable)
    
            # Create the switch and add it to the GridCal grid
            Switch1 = gce.Switch(bus_from=GCbus1, bus_to=GCbus2, name='Switch', active=switch['closed'])  # Create switch
            self.grid.add_switch(Switch1)  # Add switch to the grid
            #print("### SWITCH added : ", Switch1)
    
    def Import(self):
        self.ImportBuses()
        self.ImportLines()
        self.ImportImpedances()
        self.ImportLoads()
        self.ImportShunts()
        self.ImportExtGrid()
        self.ImportStorages()
        self.ImportSGens()
        self.ImportTrafos()
        self.ImportSwitches()
        # Return the constructed GridCal grid
        return self.grid

In [3]:
def PrintGrid_PP_GC(net_pp, net_gc):
    for _,bus in net_pp.bus.iterrows():
        print("bus pp:", bus['name'], bus.vn_kv, bus.in_service )
    for bus in net_gc.buses:
        print("bus gc:",bus.name, bus.Vnom, bus.Vmin, bus.Vmax, bus.active)        
        
    for _,line in net_pp.line.iterrows():
        print("line pp:",line['name'], line.to_bus, line.from_bus, line.r_ohm_per_km, line.x_ohm_per_km, line.c_nf_per_km, line.length_km, line.in_service )
    for line in net_gc.lines:
        print("line gc:",line.name, line.bus_to, line.bus_from, line.R, line.X, line.B, line.active)
        
    for _,ext_grid in net_pp.ext_grid.iterrows():
        print(ext_grid['name'], ext_grid.bus, ext_grid.vm_pu, ext_grid.va_degree, ext_grid.in_service )
    for _,load in net_pp.load.iterrows():
        print(load['name'], load.bus, load.p_mw, load.q_mvar, load.in_service )
    for _,trafo in net_pp.trafo.iterrows():
        print(trafo['name'], trafo.hv_bus, trafo.lv_bus, trafo.sn_mva, trafo.vn_hv_kv, trafo.vn_lv_kv, trafo.in_service )
    for _,sgen in net_pp.sgen.iterrows():
        print(sgen['name'], sgen.bus, sgen.p_mw, sgen.q_mvar, sgen.min_p_mw, sgen.max_p_mw, sgen.min_q_mvar, sgen.max_q_mvar, sgen.sn_mva, sgen.in_service )
    for _,switch in net_pp.switch.iterrows():
        print(switch['name'], switch.bus, switch.et, switch.element, switch.closed )
    for _,shunt in net_pp.shunt.iterrows():
        print(shunt['name'], shunt.bus, shunt.p_mw, shunt.q_mvar, shunt.in_service )
    for _,impedance in net_pp.impedance.iterrows():
        print(impedance['name'], impedance.from_bus, impedance.to_bus, impedance.rft_pu, impedance.xft_pu, impedance.rtf_pu, impedance.xtf_pu, impedance.sn_mva, impedance.in_service )       
    for _,storage in net_pp.storage.iterrows():
        print(storage['name'], storage.bus, storage.p_mw, storage.q_mvar, storage.soc_percent, storage.sn_mva, storage.in_service )     
        



    for gen in net_gc.generators:
        print(gen.name, gen.bus,  gen.Vset, gen.Qmin, gen.Qmax, gen.Snom, gen.active)
    for load in net_gc.loads:
        print(load.name, load.bus,  load.P, load.Q, load.active)
    for shunt in net_gc.shunts:
        print(shunt.name, shunt.bus,  shunt.G, shunt.B, shunt.active)
    for battery in net_gc.batteries:
        print(battery.name, battery.bus, battery.Pmin, battery.Pmax, battery.Qmin, battery.Qmax, battery.Sbase, battery.soc, battery.active)
    for trafo in net_gc.transformers2w:
        print(trafo.name, trafo.bus_from, trafo.bus_to, trafo.HV, trafo.LV, trafo.Sn, trafo.active)
    for switch in net_gc.switch_devices:
        print(switch.name, switch.bus_from, switch.R, switch.X, switch.bus_to, switch.active, switch.cn_from, switch.cn_to, switch.retained, switch.normal_open)

In [4]:
gridGC = FileOpen("D:\\15_Thesis-code\\DistributionNetwork_libraries\\NetworkExamples\\gridcal\\case33.gridcal").open()
for line in gridGC.lines:
    line.active = True
    
gridPP = pp.from_pickle("D:\\15_Thesis-code\\DistributionNetwork_libraries\\NetworkExamples\\pandapower\\case33bw.p")
gridPP.line.in_service = True

gridPP2GC = Panda2GridCal(gridPP).Import()

In [5]:
pp.runpp(gridPP)
print(gridPP.res_line.pl_mw.sum()*1000,"+",gridPP.res_line.ql_mvar.sum()*1000,"j")

123.29082966548187 + 87.92321168723034 j


In [6]:
options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
power_flowGC = gce.PowerFlowDriver(gridGC, options)
power_flowGC.run()

print(gridGC.name)
print('Converged:', power_flowGC.results.converged, 'error:', power_flowGC.results.error)
print(power_flowGC.results.losses.sum()/1000)


Converged: True error: 3.7443896871991456e-09
(123.29077459706916+87.9230963136972j)


In [7]:
options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
power_flowPP2GC = gce.PowerFlowDriver(gridPP2GC, options)
power_flowPP2GC.run()

print(gridGC.name)
print('Converged:', power_flowPP2GC.results.converged, 'error:', power_flowPP2GC.results.error)
print(power_flowPP2GC.results.losses.sum()*1000)


Converged: True error: 8.223360360715137e-12
(123.29082969375048+87.9232117054766j)


In [8]:
np.array(gridPP.res_bus.vm_pu)

array([1.        , 0.99709177, 0.98623793, 0.98255078, 0.97910048, 0.97104987, 0.97007704, 0.96895669, 0.96566254, 0.96523431, 0.96523367, 0.96536537, 0.96196594, 0.96076001, 0.96040567, 0.95859793, 0.95506665, 0.95395879, 0.99533161, 0.98074223, 0.97665655, 0.97292751, 0.98073572, 0.97000277, 0.96264974, 0.97005094,
       0.9687856 , 0.96362895, 0.96013625, 0.95694532, 0.95382654, 0.95327992, 0.95349821])

In [9]:
power_flowGC.results.voltage.real

array([1.        , 0.99709174, 0.98623748, 0.98255024, 0.97909989, 0.97104939, 0.97007374, 0.96895248, 0.96565661, 0.96522831, 0.96522744, 0.96535855, 0.96195883, 0.96075174, 0.96039763, 0.95859125, 0.95506014, 0.95395414, 0.99533162, 0.98074133, 0.97665409, 0.97292234, 0.9807354 , 0.97000271, 0.96264962, 0.9700506 ,
       0.96878542, 0.96362877, 0.96013615, 0.95694494, 0.95382548, 0.95327769, 0.95349492])

In [10]:
power_flowPP2GC.results.voltage.real

array([1.        , 0.99709174, 0.98623751, 0.98255029, 0.97909995, 0.97104949, 0.97007387, 0.96895266, 0.96565702, 0.96522735, 0.96522655, 0.96535779, 0.96195844, 0.96075151, 0.96039755, 0.95859119, 0.95506011, 0.95395413, 0.99533161, 0.98074123, 0.97665396, 0.97292203, 0.98073543, 0.97000275, 0.96264966, 0.9700507 ,
       0.96878551, 0.96362883, 0.96013619, 0.95694497, 0.95382549, 0.95327769, 0.95349491])

## TEST WITH A SIMBENCH NETWORW

In [11]:
sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
net3 = sb.get_simbench_net(sb_code1)
net3.switch.drop([232,234,236,238,240, 242,244,246], inplace=True)
net3.ext_grid.at[0,'name']="grid_ext"
net3.line['in_service'] = True
grid3 = Panda2GridCal(net3, 100).Import()

pp.runpp(net3)
pp_losses_all = net3.res_line.pl_mw.sum()+1j*net3.res_line.ql_mvar.sum()
pp_voltages_all = list(net3.res_bus['vm_pu'])
print(f"pandapower result with tie lines:  losses {pp_losses_all:.3f} MW, voltages {pp_voltages_all[:4]} pu")

grid3 = Panda2GridCal(net3, 100).Import()
options3 = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
power_flow3 = gce.PowerFlowDriver(grid3, options3)
power_flow3.run()
gc_losses_all = power_flow3.results.losses.sum()  #*grid3.buses[0].Vnom*grid3.Sbase/Vnom/100
gc_voltages_all = abs(power_flow3.results.voltage)

print(f"gridcal result with all connected: losses {gc_losses_all:.3f} MW, voltages {gc_voltages_all[:4]} pu")

pandapower result with tie lines:  losses 3.565-110.253j MW, voltages [1.0326290096706818, 1.0312164200952678, 1.0304622729984905, 1.0298937431826567] pu
gridcal result with all connected: losses 9.089-100.504j MW, voltages [1.18397239 1.18323385 1.18288731 1.18265614] pu


In [12]:
PrintGrid_PP_GC(gridPP, gridGC)

bus pp: 0 12.66 True
bus pp: 1 12.66 True
bus pp: 2 12.66 True
bus pp: 3 12.66 True
bus pp: 4 12.66 True
bus pp: 5 12.66 True
bus pp: 6 12.66 True
bus pp: 7 12.66 True
bus pp: 8 12.66 True
bus pp: 9 12.66 True
bus pp: 10 12.66 True
bus pp: 11 12.66 True
bus pp: 12 12.66 True
bus pp: 13 12.66 True
bus pp: 14 12.66 True
bus pp: 15 12.66 True
bus pp: 16 12.66 True
bus pp: 17 12.66 True
bus pp: 18 12.66 True
bus pp: 19 12.66 True
bus pp: 20 12.66 True
bus pp: 21 12.66 True
bus pp: 22 12.66 True
bus pp: 23 12.66 True
bus pp: 24 12.66 True
bus pp: 25 12.66 True
bus pp: 26 12.66 True
bus pp: 27 12.66 True
bus pp: 28 12.66 True
bus pp: 29 12.66 True
bus pp: 30 12.66 True
bus pp: 31 12.66 True
bus pp: 32 12.66 True
bus gc: Bus 1 12660.0 0.9 1.1 True
bus gc: Bus 2 12660.0 0.9 1.1 True
bus gc: Bus 3 12660.0 0.9 1.1 True
bus gc: Bus 4 12660.0 0.9 1.1 True
bus gc: Bus 5 12660.0 0.9 1.1 True
bus gc: Bus 6 12660.0 0.9 1.1 True
bus gc: Bus 7 12660.0 0.9 1.1 True
bus gc: Bus 8 12660.0 0.9 1.1 True
bus 

## Test with a basic network

In [13]:
def z_ohm_to_pu(r: float,
                x: float,
                c: float,
                length: float,
                Sbase: float,
                Vbase: float,
                fbase: float) -> tuple[complex, complex]:
    """_summary_

    :param r: _description_
    :param x: _description_
    :param c: _description_
    :param length: _description_
    :param Sbase: _description_
    :param Vbase: _description_
    :param fbase: _description_
    :return: _description_
    """

    zz = (r + 1j * x) * length
    z_pu = zz / (Vbase ** 2 / Sbase)

    yy = 1j * 2 * np.pi * fbase * c * 1e-9 * length
    y_pu = yy / (Sbase / Vbase ** 2)

    return (z_pu, y_pu)

In [26]:
def create_pp_network():
    # Create an empty network
    net = pp.create_empty_network()

    # Adding buses
    bus1 = pp.create_bus(net, name="Bus 1", vn_kv=110, min_vm_pu=0.9, max_vm_pu=1.1)
    bus2 = pp.create_bus(net, name="Bus 2", vn_kv=110, min_vm_pu=0.9, max_vm_pu=1.1)
    bus3 = pp.create_bus(net, name="Bus 3", vn_kv=20, min_vm_pu=0.9, max_vm_pu=1.1)
    bus4 = pp.create_bus(net, name="Bus 4", vn_kv=20, min_vm_pu=0.9, max_vm_pu=1.1)

    # Adding an external grid connection at Bus 1
    ext_grid = pp.create_ext_grid(net, name="Grid Connection", bus=bus1, vm_pu=1.02)

    # Adding line
    # It has the following parameters: r=0.642 ohm/km, x=0.083 ohm/km, c=210.0 nF/km
    line1 = pp.create_line(net, name="line 1", from_bus=bus1, to_bus=bus2, length_km=10, std_type="NAYY 4x50 SE")
    line2 = pp.create_line(net, name="line 2", from_bus=bus3, to_bus=bus4, length_km=3, std_type="NAYY 4x50 SE")
    line3 = pp.create_line(net, name="line 3", from_bus=bus2, to_bus=bus3, length_km=3, std_type="NAYY 4x50 SE")

#    trafo1 = pp.create_transformer(net, bus2, bus3, name="110kV/20kV transformer", std_type="25 MVA 110/20 kV")
    trafo1 = pp.create_transformer_from_parameters(net, bus2, bus3, name="110kV/20kV transformer", sn_mva=100,
                                                   vn_hv_kv=110, vn_lv_kv=20, vkr_percent=0.8, vk_percent=0.8, pfe_kw=1,
                                                   i0_percent=0.1)


    # Adding load to Bus 2
    load1 = pp.create_load(net, name="Load 1", bus=bus4, p_mw=30.0, q_mvar=20.0)   
    
    sgen1 = pp.create_sgen(net, name="sgen 1", bus=bus3, p_mw=4.0, q_mvar=2, min_p_mw=0, max_p_mw=4.0, min_q_mvar=0,  max_q_mvar=2, sn_mva=100, in_service=True)

    #switch1 = pp.create_switch(net, name="switch 1", bus=bus3, et='l', element=line2)

    shunt1 = pp.create_shunt(net, name="shunt 1", bus=bus2, p_mw=0, q_mvar=30)


    return net

In [27]:
def create_gc_network():
    # Create a new Grid
    grid = gce.MultiCircuit()

    # Create buses
    bus1 = gce.Bus(name='Bus 1', Vnom=float(110.))
    bus2 = gce.Bus(name='Bus 2', Vnom=float(110.))
    bus3 = gce.Bus(name='Bus 3', Vnom=float(20.))
    bus4 = gce.Bus(name='Bus 4', Vnom=float(20.))

    grid.add_bus(bus1)
    grid.add_bus(bus2)
    grid.add_bus(bus3)
    grid.add_bus(bus4)

    # Adding a line between Bus 1 and Bus 2
    zzz1, yyy1 = z_ohm_to_pu(r=0.642, x=0.083, c=210.0, length=10, Sbase=100, Vbase=110., fbase=50)
    line1 = gce.Branch(bus_from=bus1,
                      bus_to=bus2,
                      r=zzz1.real,
                      x=zzz1.imag,
                      b=yyy1.imag)
    grid.add_branch(line1)

    # Adding a line between Bus 3 and Bus 4
    zzz2, yyy2 = z_ohm_to_pu(r=0.642, x=0.083, c=210.0, length=3, Sbase=100, Vbase=20., fbase=50)
    line2 = gce.Branch(bus_from=bus3,
                      bus_to=bus4,
                      r=zzz2.real,
                      x=zzz2.imag,
                      b=yyy2.imag)
    grid.add_branch(line2)

    # Adding a line between Bus 2 and Bus 3
    zzz2, yyy2 = z_ohm_to_pu(r=0.642, x=0.083, c=210.0, length=3, Sbase=100, Vbase=110., fbase=50)
    line3 = gce.Branch(bus_from=bus2,
                      bus_to=bus3,
                      r=zzz2.real,
                      x=zzz2.imag,
                      b=yyy2.imag)
    #grid.add_branch(line3)

    Transformer1 = gce.Transformer2W(bus_from=bus2, bus_to=bus3, name='Transformer 1', HV=110, LV=20, nominal_power=100,
                                     copper_losses=1, r=0.01, x=0.1)
    grid.add_transformer2w(Transformer1)

    # Adding a load at Bus 2
    load = gce.Load(P=30.0, Q=20.0)  # Active and reactive power in MW and MVar
    grid.add_load(bus4, load)  # Adding the load to the bus

    # Adding a generator at Bus 1
    gen = gce.Generator(name="ext_grid", P=1.0, vset=1.02)  # Active power in MW and voltage set point in p.u.
    grid.add_generator(bus1, gen)  # Adding the generator to the bus
    
    gen = gce.Generator(name="sgen1", P=4.0, is_controlled=False)  # Active power in MW and voltage set point in p.u.
    grid.add_generator(bus3, gen)  # Adding the generator to the bus
    
    # Adding a generator at Switch 1
    #switch1 = gce.Switch(name="switch1", bus_from=bus3,
    #                  bus_to=bus4,
    #                  r=zzz2.real,
    #                  x=zzz2.imag,
    #                  active = True,
    #                  normal_open=False)
    #grid.add_switch(switch1)  # Adding the generator to the bus

    # adding a shunt to bus2 
    Sbase = 100
    VbaseShunt = 110
    zbase = VbaseShunt ** 2 / Sbase  # Calculate base impedance
    gu = 0 / VbaseShunt ** 2 * zbase
    bu = 30 / VbaseShunt ** 2 * zbase
    shunt1 = gce.Shunt(name="shunt1", G=gu, B=bu, active=True)
    grid.add_shunt(bus2, shunt1)

    return grid

In [31]:
netPP = create_pp_network()
pp.runpp(netPP)
print(f"pandapower result created:\t\t", [f"{num:.4f}" for num in (netPP.res_bus['vm_pu'])])
print(f"losses:{netPP.res_line.pl_mw.sum()*1000:.4f} MW")

pandapower result created:		 ['1.0200', '0.9977', '0.9957', '0.7931']
losses:11642.6392 MW


In [33]:
netGC = Panda2GridCal(netPP, 100).Import()
options3 = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
power_flowGC = gce.PowerFlowDriver(netGC, options3)
power_flowGC.run()
gc_losses_all = power_flowGC.results.losses.sum()  #*grid3.buses[0].Vnom*grid3.Sbase/Vnom/100
gc_voltages_all = abs(power_flowGC.results.voltage)

print(f"gridcal result converted: \t\t {[f"{num:.4f}" for num in gc_voltages_all]}")
print(f"losses {gc_losses_all.real*1000:.3f} MW")

gridcal result converted: 		 ['1.0200', '1.0000', '0.9939', '0.7906']
losses 11011.346 MW


In [30]:
Vnom=110

gc_grid = create_gc_network()
options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
power_flow = gce.PowerFlowDriver(gc_grid, options)
power_flow.run()
vm_gc = abs(power_flow.results.voltage)
gc_losses = power_flow.results.losses.sum()*gc_grid.buses[0].Vnom*gc_grid.Sbase/Vnom/100
print(f"gridcal result created: \t\t {[f"{num:.4f}" for num in vm_gc]}")
print(f"losses: {gc_losses.real*1000}")

gridcal result created: 		 ['1.0200', '0.9993', '0.9762', '0.7662']
losses: 11685.213320589031
